In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import albumentations
import pandas as pd
import numpy as np
import shutil
import os
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
class CFG():
    img_size = 299
    seed = 42
    classes = [
        'complex', 
        'frog_eye_leaf_spot', 
        'powdery_mildew', 
        'rust', 
        'scab',
        'healthy']
    batch_size=32

In [ ]:
train_dir='../input/plant-pathology-2021-fgvc8/test_images'

In [ ]:

test_img = '../input/plant-pathology-2021-fgvc8/test_images'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission.head()

In [ ]:
submission['labels'] = submission['labels'].apply(lambda s: s.split(' '))
submission.head()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
''' ImageDataGenerator參考網頁-> https://keras.io/zh/preprocessing/image/ '''
datagen = ImageDataGenerator(
    rescale = 1/255.,
    rotation_range = 15,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    zoom_range = 0.2,
    shear_range=0.3, #（以弧度逆时针方向剪切角度）。
    horizontal_flip=True,
    vertical_flip=True,
    validation_split= 0.2,)

In [ ]:
test_generator = datagen.flow_from_dataframe(
                  submission,
                  directory = test_img,
                  x_col = 'image',
                  y_col = 'labels',
                  class_mode = "categorical",
                  batch_size=4,
                  target_size = (CFG.img_size,CFG.img_size),
                  color_mode="rgb",
                  shuffle = False,
                  seed = CFG.seed)

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
#加載所有模型並將它們存儲在列表中
model_files = os.listdir('../input/xception-skf')
models = []

for model_file in model_files:
  print(f'loading model {model_file}')
  models.append(keras.models.load_model(f'../input/xception-skf/{model_file}', compile = True))

In [ ]:
predict=[]
for model in models:
    pred = model.predict(test_generator)
    pred =np.around(pred, decimals=3, out=None)
    predict.append( pred)

In [ ]:
predict

In [ ]:
import numpy as np

predict= np.mean(predict, axis=0)

In [ ]:
print(predict)

In [ ]:
np.around(predict, decimals=3, out=None)

In [ ]:
perdict = (predict>0.33)
n_label = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
answer = []

for i in range(perdict.shape[0]):
    temp = []
    for j, k in enumerate(n_label):
        if perdict[i, j]:
            temp.append(k)
    answer.append(temp)
    
answer = [' '.join(n) for n in answer]

In [ ]:
answer

In [ ]:
submission['labels'] = np.array(answer)
submission

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
# !cp /content/B4.h5 /content/B4_0.h5

In [ ]:
# !mv /content/B4_0.h5 /content/drive/MyDrive/apple